In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os 
os.chdir('/content/drive/MyDrive/vietnamesesentiment ')

In [ ]:
import pandas as pd 
df = pd.read_csv('combine.csv')
df.head()

,sentence,label
0,Tao hít không khí thui cũng mập địt 😂😂,1
1,Nguyễn Hồng DiệuÝ má chị hằng mập địt vler,0
2,Lê Viết Quang nghe rất địt mẹ mày,0
3,Thành Trần địt phải chả bạn ơi nó lấy lãy 50% ...,0
4,Vay của home còn đỡ Bọn f.e t đéo vay bgio mu...,0


In [ ]:
df['label'].value_counts()

2    16822
0    15775
1      515
Name: label, dtype: int64

In [ ]:
df.shape

(33112, 2)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from collections import Counter

import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [ ]:
# from underthesea.word_tokenize.regex_tokenize import tokenize

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from collections import Counter

import numpy as np
# import unidecode
import pandas as pd
import string
import re
import os
import shutil
import pickle 

In [ ]:
df = pd.read_csv('combine.csv')
X = df['sentence'].values
Y = df['label'].values
index = np.arange(X.shape[0])
np.random.shuffle(index)
X = X[index]
Y = Y[index]

In [ ]:
def transform(x):
  return [s.lower() for s in x]
# def remove_tone(x):
#   return [unidecode.unidecode(s) for s in x] 
def remove_duplicate(x):
  result = []
  for s in x:
      s = re.sub(r'([a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      s = re.sub(r'([a-z][a-z])\1+', lambda m: m.group(1), s, flags=re.IGNORECASE)
      result.append(s)
  return result

In [ ]:
X = transform(X)
X = remove_duplicate(X)

In [ ]:
X[0]

'không hiểu sao lại có quán vừa sang vừa đẹp mà giá lại rẻ thế này\n\xa0 bước vào mà đối lập không gian bên ngoài\nquán thiết kế rất ok , không gian tạo vẻ ấm cúng , rất thích ánh sáng vàng trong quán\ntiếc là xa quá không thì còn ghé thường xuyên hơn\n\xa0 chỉ góp ý thêm chút là nên gắn điều hòa hoặc quạt ở các bàn vì thực sự nhiều khi rất nóng mà không biết làm sao'

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,4),norm='l2', min_df=2,max_features=6000)
X = vectorizer.fit_transform(X)

In [ ]:
X.shape

(33112, 6000)

In [ ]:
XGB = XGBClassifier(eta=0.1,max_depth=6,objective='multi:softmax',scale_pos_weight=1)

In [ ]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]

XGB.fit(X_train, Y_train)

train_acc = (XGB.predict(X_train) == Y_train).astype(int).mean()
test_acc = (XGB.predict(X_test) == Y_test).astype(int).mean()

print("Train acc: ", train_acc, " Test acc: ", test_acc)

Train acc:  0.8847784700588439  Test acc:  0.8334


In [ ]:
X_train = X[:-10000]
Y_train = Y[:-10000]
X_test = X[-10000:]
Y_test = Y[-10000:]

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [ ]:
def hyperParameterTuning(X_train, Y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['multi:softmax'],
        'scale_pos_weight': [1]
    }

    xgb_model = XGBClassifier()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,Y_train)
    return gsearch.best_params_

In [ ]:
print(hyperParameterTuning(X_train,Y_train))

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 21.0min
